# Laplacian Convolutional Representation (LCR)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z, w, lmbda, denominator):
    T = z.shape[0]
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.0518124484350609
3.9666053593025814

20
0.02093354304049163
1.9572561575393177

30
0.019602826189092003
1.8647930446427807

40
0.01897075701509782
1.8162100675813548

50
0.018577455882051605
1.7854006049039857

60
0.018303271911143042
1.764066142385454

70
0.01810026047420505
1.7485225920609393

80
0.01794479397010613
1.7368063389380104

90
0.017822875429319252
1.7277486451874875

100
0.01772542861877724
1.7206100005211489

Running time: 1756 seconds.


In [3]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.5
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.21812494934602628
16.862135010556315

20
0.02500976176059953
2.1932578447042435

30
0.02129041028597947
2.005472753775813

40
0.02046884471575034
1.9535692806948164

50
0.020029928408220644
1.9233017896977365

60
0.01974353502762083
1.9029881041883425

70
0.019537833688258587
1.8883446828570478

80
0.019382243600542505
1.8773380923090524

90
0.0192616503701335
1.8688656188385686

100
0.019166286979421902
1.8622224750675556

Running time: 1742 seconds.


In [4]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.7
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.6581365424140968
50.22503566747938

20
0.06690381395850381
5.03946546307415

30
0.02736811657769783
2.390471394052395

40
0.024006819663374184
2.2388148401391357

50
0.023212043099511796
2.2049885108054665

60
0.02285005706341285
2.1878674118027326

70
0.022633091033711743
2.176941103926788

80
0.022484844917928064
2.1691308237013907

90
0.022375794552218515
2.1632357758437912

100
0.02229237580897109
2.1586298426493715

Running time: 1718 seconds.


In [5]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.820489993344388
59.72595274971853

20
0.26057797061695187
19.744136356485246

30
0.07744138334993228
5.724489779074404

40
0.046857154369039596
3.628871437827636

50
0.03913262155707002
3.2498714579167283

60
0.0361817145785277
3.1256267781116107

70
0.03492649130578826
3.079833493126235

80
0.03445040054287662
3.0638762712207037

90
0.03425143560028867
3.058707806104131

100
0.034150762570443145
3.0565379457542985

Running time: 1695 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>